In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import csv


In [17]:
#!pip install  selenium beautifulsoup4 pandas requests
#!pip install undetected-chromedriver

In [18]:
# 1. Definir la URL de interés
url = "https://www.inmuebles24.com/departamentos-en-renta-q-cdmx.html"

In [23]:
# -*- coding: utf-8 -*-

import csv
import time
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import os

# 1. Definir la URL de interés y la ruta del archivo de salida
url = "https://www.inmuebles24.com/departamentos-en-renta-q-cdmx.html"
csv_filename = "/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/inmuebles.csv"

# 2. Crear una lista para almacenar los datos de los inmuebles.
# Se declara aquí para que siempre exista, incluso si ocurre un error.
inmuebles_data = []

# 3. Configurar el WebDriver de Selenium
# Usa undetected_chromedriver para evitar los sistemas de detección de bots
# Aumentamos el timeout para evitar errores de desconexión
try:
    print("Iniciando el navegador web sin detección...")
    driver = uc.Chrome(options=uc.ChromeOptions(), timeout=240)
    driver.maximize_window()
except WebDriverException as e:
    print(f"Error al iniciar el navegador: {e}")
    print("Asegúrate de tener la librería 'undetected-chromedriver' instalada.")
    exit()

try:
    # 4. Abrir la URL
    print(f"Abriendo la página: {url}")
    driver.get(url)
    
    # --- PASO DE DEPURACIÓN: Comprobar la URL final y tomar una captura de pantalla ---
    print(f"URL actual del navegador: {driver.current_url}")
    screenshot_path = "pagina_inmuebles.png"
    driver.save_screenshot(screenshot_path)
    print(f"Captura de pantalla guardada en: {os.path.abspath(screenshot_path)}")
    # ------------------------------------------------------------------------------------
    
    # Usar una espera explícita para asegurar que el contenedor principal esté presente
    wait = WebDriverWait(driver, 60)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.postingsList-module__postings-container')))
    
    print("El contenedor de resultados se ha cargado correctamente.")

    # 5. Esperar a que al menos un listado de propiedad sea visible
    # Usamos un selector más específico de la imagen que proporcionaste.
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.postingCardLayout-module__posting-card-layout')))
    
    print("Los listados de propiedades han comenzado a cargar.")

    # 6. Encontrar todos los listados de propiedades
    listings = driver.find_elements(By.CSS_SELECTOR, '.postingCardLayout-module__posting-card-layout')
    print(f"Se encontraron {len(listings)} listados de propiedades.")

    # 7. Iterar a través de cada listado y extraer la información
    for index, listing in enumerate(listings):
        print(f"\n--- Propiedad {index + 1} ---")

        # Inicializar un diccionario para la propiedad actual
        inmueble = {}
        
        # Extraer el título y el enlace
        try:
            # Buscamos el div con la descripción y dentro el enlace 'a'
            title_container = listing.find_element(By.CSS_SELECTOR, '.postingCard-module__posting-description')
            # Ahora extraemos el texto del contenedor para usarlo como título.
            inmueble['Título'] = title_container.text.strip()
            title_element = title_container.find_element(By.TAG_NAME, 'a')
            inmueble['Enlace'] = title_element.get_attribute('href')
            print(f"Título: {inmueble['Título']}")
            print(f"Enlace: {inmueble['Enlace']}")
        except Exception as e:
            inmueble['Título'] = 'No disponible'
            inmueble['Enlace'] = 'No disponible'
            print(f"Título y Enlace no encontrados. Error: {e}")

        # Extraer el precio
        try:
            # El precio se encuentra dentro del contenedor 'postingCard-module__posting-prices-and-publisher'
            # y tiene la clase 'posting-price'. Buscamos directamente el elemento de precio.
            price_element = listing.find_element(By.CSS_SELECTOR, 'div[data-qa="POSTING_CARD_PRICE"]')
            inmueble['Precio'] = price_element.text.strip()
            print(f"Precio: {inmueble['Precio']}")
        except Exception as e:
            inmueble['Precio'] = 'No disponible'
            print(f"Precio no encontrado. Error: {e}")

        # Extraer la dirección
        try:
            address_element = listing.find_element(By.CSS_SELECTOR, '.postingLocations-module__location-block')
            inmueble['Dirección'] = address_element.text.strip()
            print(f"Dirección: {inmueble['Dirección']}")
        except Exception as e:
            inmueble['Dirección'] = 'No disponible'
            print(f"Dirección no encontrada. Error: {e}")

         # Extraer las características
        try:
            # Ahora el selector de características es más específico
            features = listing.find_elements(By.CSS_SELECTOR, 'span.postingMainFeatures-module__posting-main-features-listing')
            features_list = [feature.text.strip() for feature in features]
            inmueble['Características'] = ', '.join(features_list)
            for feature in features_list:
                print(f"  - Característica: {feature}")
        except Exception as e:
            inmueble['Características'] = 'No disponible'
            print(f"Características no encontradas. Error: {e}")

        # Añadir el diccionario de la propiedad a la lista de datos
        inmuebles_data.append(inmueble)
    
    # 8. Escribir los datos en un archivo CSV
    # Asegurarse de que el directorio exista antes de escribir el archivo
    os.makedirs(os.path.dirname(csv_filename), exist_ok=True)
    
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Título', 'Enlace', 'Precio', 'Dirección', 'Características']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Escribe la fila de encabezados
        writer.writerows(inmuebles_data) # Escribe todas las filas de datos
    
    print(f"\nDatos guardados exitosamente en '{csv_filename}'.")

except TimeoutException:
    print("Se superó el tiempo de espera. La página o los elementos de resultados no se cargaron a tiempo.")
except Exception as e:
    print(f"Ocurrió un error inesperado durante el scraping: {e}")
finally:
    # 9. Cerrar el navegador al finalizar
    print("Cerrando el navegador.")
    driver.quit()

print("\nScraping completado.")


Iniciando el navegador web sin detección...
Abriendo la página: https://www.inmuebles24.com/departamentos-en-renta-q-cdmx.html
URL actual del navegador: https://www.inmuebles24.com/departamentos-en-renta-q-cdmx.html
Captura de pantalla guardada en: /Users/renataramirez/Documents/mlops-repo/notebooks/01.Extraction/pagina_inmuebles.png
El contenedor de resultados se ha cargado correctamente.
Los listados de propiedades han comenzado a cargar.
Se encontraron 30 listados de propiedades.

--- Propiedad 1 ---
Título: Sé el primero en experimentar la vida de lujo en Gran Ciudad Nuevo Polanco. Departamentos en renta a estrenar con amenidades inigualables. Reserva tu departamento hoy mismo y disfruta de beneficios especiales. Deja tu información de contacto, ¡nuestro equipo se pondrá en contacto contigo en breve! Elige tu nuevo departamento: Estudios: desde $24, 300 mxn. Departamento de 1 Recámara: desde $27, 500 mxn. Departamento de 2 Recámaras: desde $36, 000 mxn. *Contamos con espacio de est

In [20]:
csv_filename

'/Users/renataramirez/Documents/mlops-repo/data/raw/webscrapping/inmuebles.csv'

In [21]:
len(inmuebles_data)

30

In [22]:
inmuebles_data

[{'Título': '',
  'Enlace': 'https://www.inmuebles24.com/propiedades/desarrollo/ememvein-gran-ciudad-nuevo-polanco-144968760.html',
  'Precio': 'MN 24,300',
  'Dirección': 'Río San Joaquín 498, Ampliación Granada, CDMX\nPolanco, Miguel Hidalgo',
  'Características': '329 un., 1 a 2 rec.'},
 {'Título': '',
  'Enlace': 'https://www.inmuebles24.com/propiedades/clasificado/alclapin-departamento-en-el-centro-de-la-cdmx-146299506.html',
  'Precio': 'MN 30,000',
  'Dirección': 'Juarez, Centro, Cuauhtémoc, Cdmx al 100\nCentro , Cuauhtémoc',
  'Características': '59 m² lote, 2 rec., 1 baño'},
 {'Título': '',
  'Enlace': 'https://www.inmuebles24.com/propiedades/desarrollo/ememvein-departamentos-en-renta-en-paseo-de-la-reforma-cdmx-64699580.html',
  'Precio': '',
  'Dirección': '',
  'Características': ', '},
 {'Título': '',
  'Enlace': 'https://www.inmuebles24.com/propiedades/clasificado/alclapin-departamento-en-renta-en-la-colonia-roma-norte-147306224.html',
  'Precio': '',
  'Dirección': '',
 